In [1]:
!pip install dlib

In [2]:
import cv2
import numpy as np
import dlib
from imutils import face_utils

# Initialize the camera
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Camera could not be opened.")
    exit()

# Load the face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define eye and mouth indices from the 68 facial landmarks
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# Status flags and counters
sleep = 0
drowsy = 0
active = 0
blinks = 0
yawns = 0
status = ""
color = (0, 0, 0)
alarm_on = False

# Define constants
EAR_THRESHOLD = 0.25
EAR_CONSECUTIVE_FRAMES = 3
MAR_THRESHOLD = 0.6
MAR_CONSECUTIVE_FRAMES = 15

def compute(ptA, ptB):
    """Compute the Euclidean distance between two points."""
    dist = np.linalg.norm(ptA - ptB)
    return dist

def eye_aspect_ratio(eye):
    """Calculate the Eye Aspect Ratio (EAR) to determine eye openness."""
    A = compute(eye[1], eye[5])
    B = compute(eye[2], eye[4])
    C = compute(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    """Calculate the Mouth Aspect Ratio (MAR) to detect yawns."""
    A = compute(mouth[2], mouth[10])
    B = compute(mouth[4], mouth[8])
    C = compute(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

def sound_alarm(path):
    """Play or stop the alarm sound."""
    try:
        import winsound
        if path:
            winsound.PlaySound(path, winsound.SND_ASYNC | winsound.SND_LOOP)
        else:
            winsound.PlaySound(None, winsound.SND_ASYNC)  # Stop the alarm
    except ImportError:
        pass  # Handle the absence of winsound

# Variables for blink and yawn detection
eye_closed = False
blink_detection_frames = 0
yawn_detection_frames = 0

# Main loop for video capture and processing
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) > 0:  # Only proceed if a face is detected
        for face in faces:
            landmarks = predictor(gray, face)
            landmarks = face_utils.shape_to_np(landmarks)

            # Extract left and right eye and mouth coordinates
            leftEye = landmarks[lStart:lEnd]
            rightEye = landmarks[rStart:rEnd]
            mouth = landmarks[mStart:mEnd]

            # Calculate the EAR for both eyes
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # Calculate MAR for the mouth
            mar = mouth_aspect_ratio(mouth)

            # Debugging: Print EAR and MAR values
            print(f"Left EAR: {leftEAR:.2f}, Right EAR: {rightEAR:.2f}, Average EAR: {ear:.2f}")
            print(f"MAR: {mar:.2f}")

            # Check if eyes are closed or open
            if ear < EAR_THRESHOLD:
                blink_detection_frames += 1
                if blink_detection_frames >= EAR_CONSECUTIVE_FRAMES:
                    if not eye_closed:
                        blinks += 1
                        eye_closed = True
                        status = "Drowsy!"
                        color = (0, 0, 255)  # Red color for drowsiness indication
                        if not alarm_on:
                            sound_alarm('alarm.wav')
                            alarm_on = True
            else:
                if eye_closed:
                    eye_closed = False
                blink_detection_frames = 0

            # Yawn detection
            if mar > MAR_THRESHOLD:
                yawn_detection_frames += 1
                if yawn_detection_frames >= MAR_CONSECUTIVE_FRAMES:
                    yawns += 1
                    status = "YAWNING !!!"
                    color = (255, 0, 255)  # Purple color for yawning indication
                    if not alarm_on:
                        sound_alarm('alarm.wav')
                        alarm_on = True
            else:
                yawn_detection_frames = 0

            # Update status based on eye closure and yawn detection
            if ear < EAR_THRESHOLD and blink_detection_frames >= EAR_CONSECUTIVE_FRAMES:
                status = "SLEEPING !!!"
                color = (255, 0, 0)  # Red color for sleep indication
                if not alarm_on:
                    sound_alarm('alarm.wav')
                    alarm_on = True
            elif yawn_detection_frames >= MAR_CONSECUTIVE_FRAMES:
                status = "Drowsy!"
                color = (0, 0, 255)  # Red color for drowsiness indication
                if not alarm_on:
                    sound_alarm('alarm.wav')
                    alarm_on = True
            else:
                if alarm_on:
                    sound_alarm(None)  # Stop the alarm
                    alarm_on = False
                drowsy = 0
                sleep = 0
                active += 1
                if active > 6:
                    status = "Active :)"
                    color = (0, 255, 0)  # Green color for active state

            # Display status, blink count, and yawn count on the frame
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(frame, f"Blinks: {blinks}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"Yawns: {yawns}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            # Draw rectangles around the eyes
            leftEyeRect = cv2.boundingRect(cv2.convexHull(leftEye))
            rightEyeRect = cv2.boundingRect(cv2.convexHull(rightEye))
            cv2.rectangle(frame, (leftEyeRect[0], leftEyeRect[1]), 
                              (leftEyeRect[0] + leftEyeRect[2], leftEyeRect[1] + leftEyeRect[3]), 
                              (255, 0, 0), 2)
            cv2.rectangle(frame, (rightEyeRect[0], rightEyeRect[1]), 
                              (rightEyeRect[0] + rightEyeRect[2], rightEyeRect[1] + rightEyeRect[3]), 
                              (255, 0, 0), 2)

            # Draw contours around the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            # Visualize all facial landmarks (optional)
            for n in range(0, 68):
                (x, y) = landmarks[n]
                cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)

    # Show the main webcam feed with only the eye regions
    cv2.imshow("Webcam Feed", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Esc key to exit
        sound_alarm(None)  # Ensure alarm is stopped
        break

cap.release()
cv2.destroyAllWindows()
import cv2
import numpy as np
import dlib
from imutils import face_utils

# Initialize the camera
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Camera could not be opened.")
    exit()

# Load the face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define eye and mouth indices from the 68 facial landmarks
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# Status flags and counters
sleep = 0
drowsy = 0
active = 0
blinks = 0
yawns = 0
status = ""
color = (0, 0, 0)
alarm_on = False

# Define constants
EAR_THRESHOLD = 0.25
EAR_CONSECUTIVE_FRAMES = 3
MAR_THRESHOLD = 0.6
MAR_CONSECUTIVE_FRAMES = 15

def compute(ptA, ptB):
    """Compute the Euclidean distance between two points."""
    dist = np.linalg.norm(ptA - ptB)
    return dist

def eye_aspect_ratio(eye):
    """Calculate the Eye Aspect Ratio (EAR) to determine eye openness."""
    A = compute(eye[1], eye[5])
    B = compute(eye[2], eye[4])
    C = compute(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    """Calculate the Mouth Aspect Ratio (MAR) to detect yawns."""
    A = compute(mouth[2], mouth[10])
    B = compute(mouth[4], mouth[8])
    C = compute(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

def sound_alarm(path):
    """Play or stop the alarm sound."""
    try:
        import winsound
        if path:
            winsound.PlaySound(path, winsound.SND_ASYNC | winsound.SND_LOOP)
        else:
            winsound.PlaySound(None, winsound.SND_ASYNC)  # Stop the alarm
    except ImportError:
        pass  # Handle the absence of winsound

# Variables for blink and yawn detection
eye_closed = False
blink_detection_frames = 0
yawn_detection_frames = 0

# Main loop for video capture and processing
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) > 0:  # Only proceed if a face is detected
        for face in faces:
            landmarks = predictor(gray, face)
            landmarks = face_utils.shape_to_np(landmarks)

            # Extract left and right eye and mouth coordinates
            leftEye = landmarks[lStart:lEnd]
            rightEye = landmarks[rStart:rEnd]
            mouth = landmarks[mStart:mEnd]

            # Calculate the EAR for both eyes
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # Calculate MAR for the mouth
            mar = mouth_aspect_ratio(mouth)

            # Debugging: Print EAR and MAR values
            print(f"Left EAR: {leftEAR:.2f}, Right EAR: {rightEAR:.2f}, Average EAR: {ear:.2f}")
            print(f"MAR: {mar:.2f}")

            # Check if eyes are closed or open
            if ear < EAR_THRESHOLD:
                blink_detection_frames += 1
                if blink_detection_frames >= EAR_CONSECUTIVE_FRAMES:
                    if not eye_closed:
                        blinks += 1
                        eye_closed = True
                        status = "Drowsy!"
                        color = (0, 0, 255)  # Red color for drowsiness indication
                        if not alarm_on:
                            sound_alarm('alarm.wav')
                            alarm_on = True
            else:
                if eye_closed:
                    eye_closed = False
                blink_detection_frames = 0

            # Yawn detection
            if mar > MAR_THRESHOLD:
                yawn_detection_frames += 1
                if yawn_detection_frames >= MAR_CONSECUTIVE_FRAMES:
                    yawns += 1
                    status = "YAWNING !!!"
                    color = (255, 0, 255)  # Purple color for yawning indication
                    if not alarm_on:
                        sound_alarm('alarm.wav')
                        alarm_on = True
            else:
                yawn_detection_frames = 0

            # Update status based on eye closure and yawn detection
            if ear < EAR_THRESHOLD and blink_detection_frames >= EAR_CONSECUTIVE_FRAMES:
                status = "SLEEPING !!!"
                color = (255, 0, 0)  # Red color for sleep indication
                if not alarm_on:
                    sound_alarm('alarm.wav')
                    alarm_on = True
            elif yawn_detection_frames >= MAR_CONSECUTIVE_FRAMES:
                status = "Drowsy!"
                color = (0, 0, 255)  # Red color for drowsiness indication
                if not alarm_on:
                    sound_alarm('alarm.wav')
                    alarm_on = True
            else:
                if alarm_on:
                    sound_alarm(None)  # Stop the alarm
                    alarm_on = False
                drowsy = 0
                sleep = 0
                active += 1
                if active > 6:
                    status = "Active :)"
                    color = (0, 255, 0)  # Green color for active state

            # Display status, blink count, and yawn count on the frame
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(frame, f"Blinks: {blinks}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"Yawns: {yawns}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            # Draw rectangles around the eyes
            leftEyeRect = cv2.boundingRect(cv2.convexHull(leftEye))
            rightEyeRect = cv2.boundingRect(cv2.convexHull(rightEye))
            cv2.rectangle(frame, (leftEyeRect[0], leftEyeRect[1]), 
                              (leftEyeRect[0] + leftEyeRect[2], leftEyeRect[1] + leftEyeRect[3]), 
                              (255, 0, 0), 2)
            cv2.rectangle(frame, (rightEyeRect[0], rightEyeRect[1]), 
                              (rightEyeRect[0] + rightEyeRect[2], rightEyeRect[1] + rightEyeRect[3]), 
                              (255, 0, 0), 2)

            # Draw contours around the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            # Visualize all facial landmarks (optional)
            for n in range(0, 68):
                (x, y) = landmarks[n]
                cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)

    # Show the main webcam feed with only the eye regions
    cv2.imshow("Webcam Feed", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Esc key to exit
        sound_alarm(None)  # Ensure alarm is stopped
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'dlib'

In [ ]:
import sys
print(sys.executable)

c:\Users\Franklin Jerold F\AppData\Local\Programs\Python\Python312\python.exe


In [ ]:
!pip install dlib


'C:\Users\Franklin' is not recognized as an internal or external command,
operable program or batch file.
